In [ ]:

import random
import time

# Directions for moving the blank space
DIRECTIONS = {
    (-1, 0): "Up",
    (1, 0): "Down",
    (0, -1): "Left",
    (0, 1): "Right"
}


def is_solvable(state, n):
    flat_state = [tile for row in state for tile in row if tile != 0]
    inversions = sum(
        flat_state[i] > flat_state[j] for i in range(len(flat_state)) for j in range(i + 1, len(flat_state)))
    return inversions % 2 == 0


def goal_state(n):
    return [list(range(i * n, (i + 1) * n)) for i in range(n)]


def find_blank(state):
    for i, row in enumerate(state):
        for j, tile in enumerate(row):
            if tile == 0:
                return i, j


def neighbors(state):
    n = len(state)
    blank_i, blank_j = find_blank(state)
    possible_moves = []

    for di, dj in DIRECTIONS.keys():
        new_i, new_j = blank_i + di, blank_j + dj
        if 0 <= new_i < n and 0 <= new_j < n:
            new_state = [row[:] for row in state]
            new_state[blank_i][blank_j], new_state[new_i][new_j] = new_state[new_i][new_j], new_state[blank_i][blank_j]
            possible_moves.append((new_state, DIRECTIONS[(di, dj)]))

    return possible_moves


def dfs_revisit(state, goal, depth_limit):
    """Modified DFS with revisit check to track all visited states."""
    stack = [(state, [], 0)]
    visited = set()
    visited.add(tuple(tuple(row) for row in state))

    max_states_stored = 1

    while stack:
        current_state, path, depth = stack.pop()

        # Track max number of states stored (visited and current states in the stack)
        max_states_stored = max(max_states_stored, len(visited))

        if current_state == goal:
            return path, len(path), max_states_stored

        if depth < depth_limit:
            for neighbor, action in neighbors(current_state):
                neighbor_tuple = tuple(tuple(row) for row in neighbor)
                if neighbor_tuple not in visited:
                    visited.add(neighbor_tuple)
                    stack.append((neighbor, path + [action], depth + 1))

    return None, 0, max_states_stored  # No solution found


def generate_easy_puzzle(n):
    """Generate a randomized easy-to-solve puzzle for n x n grid."""
    puzzle = goal_state(n)
    moves = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # Possible moves

    # Perform a random sequence of moves
    for _ in range(random.randint(5, 10)):  # Random number of moves
        random_move = random.choice(moves)
        blank_i, blank_j = find_blank(puzzle)
        new_i, new_j = blank_i + random_move[0], blank_j + random_move[1]

        if 0 <= new_i < n and 0 <= new_j < n:
            puzzle[blank_i][blank_j], puzzle[new_i][new_j] = puzzle[new_i][new_j], puzzle[blank_i][blank_j]

    return puzzle


def run_puzzle_solver_dfs_revisit(n_values=[3, 4, 5, 6], repetitions=10):
    for n in n_values:
        print(f"\nSolving {n ** 2 - 1}-puzzle (n = {n}):")
        solution_depths = []
        max_memory_usages = []

        for i in range(repetitions):
            depth_limit = random.randint(5, 30)
            initial = generate_easy_puzzle(n)  # Generate easier initial states
            goal = goal_state(n)

            print(f"\nRun {i + 1}:")
            print("Initial State:")
            for row in initial:
                print(row)
            print("Goal State:")
            for row in goal:
                print(row)

            start_time = time.time()
            actions, solution_depth, max_states_stored = dfs_revisit(initial, goal, depth_limit=depth_limit)
            elapsed_time = time.time() - start_time

            if actions is not None:
                print(f"Solution depth: {solution_depth}")
                print(f"Max memory usage (states stored): {max_states_stored}")
                print(f"Solution sequence of actions: {', '.join(actions)}")
            else:
                print("No solution found.")

            solution_depths.append(solution_depth)
            max_memory_usages.append(max_states_stored)

        print(f"\nStatistics for {n}-puzzle:")
        print(
            f"Solution depth: Min = {min(solution_depths)}, Max = {max(solution_depths)}, Avg = {sum(solution_depths) / len(solution_depths):.1f}")
        print(
            f"Max memory usage(states stored): Min = {min(max_memory_usages)}, Max = {max(max_memory_usages)}, Avg = {sum(max_memory_usages) / len(max_memory_usages):.1f}")


# Run the solver for puzzles of size n = 3, 4, 5, 6 with 10 repetitions each
run_puzzle_solver_dfs_revisit()
